In [1]:
import numpy as np
from scipy.spatial import distance
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
import sys
from itertools import product

name = "ioni_gas"

location_grid_coords = "../data/FIRE/grid-coords.npy"
location_non_empty_voxel_coords = "../data/FIRE/non-empty-coords-"+name+".npy"
# name = location_non_empty_voxel_coords[30:][0:location_non_empty_voxel_coords[30:].rfind(".")]
location_save_grid_ids = "../data/FIRE/grid-ids--"+name+".npy"

print(f">> loading in voxel grid coordinates...")
grid_c = np.load(location_grid_coords)
print(f">> loaded in voxel grid coordinates from {location_grid_coords}")

print(f">> loading in non-empty voxel coordinates...")
non_empty_points = np.load(location_non_empty_voxel_coords)
non_empty_points_length = len(non_empty_points)
print(f">> loaded in non-empty voxel coordinates from {location_non_empty_voxel_coords}")

print(f">> {name}")

>> loading in voxel grid coordinates...
>> loaded in voxel grid coordinates from ../data/FIRE/grid-coords.npy
>> loading in non-empty voxel coordinates...
>> loaded in non-empty voxel coordinates from ../data/FIRE/non-empty-coords-ioni_gas.npy
>> ioni_gas


In [2]:
current_min_distances = np.ones(len(grid_c), dtype=int) * 20**10
grid_ids = np.empty(len(grid_c), dtype=int)

def process_interval(indexes):
    start, end, points_offset_start, points_offset_end = indexes[0][0], indexes[0][1], indexes[1][0], indexes[1][1]
    print(f">> grid batch: [{start} - {end}] \t point batch: [{points_offset_start} - {points_offset_end}]", end="\r")
    # calculate all of the distances between this batch of grid points and filled voxel points
    distances = distance.cdist(grid_c[start:end], non_empty_points[points_offset_start:points_offset_end])
    # identify the nearest voxels to each grid point
    proposed_min_distances = np.min(distances, axis=1)
    proposed_min_ids = np.argmin(distances, axis=1) + points_offset_start

    improved_indexes = proposed_min_distances<current_min_distances[start:end]
    
    current_min_distances[start:end][improved_indexes] = proposed_min_distances[improved_indexes]
    grid_ids[start:end][improved_indexes] = proposed_min_ids[improved_indexes]

In [5]:
from scipy.spatial import cKDTree
# t0 = timeit.default_timer()
tree_points = cKDTree(non_empty_points)
indices = tree_points.query(grid_c)[1]
# tf = timeit.default_timer()
# print(tf - t0)

In [21]:
np.save("../data/FIRE/grid-ids--ioni_gas-cKDTree.npy", indices)

In [20]:
np.sum(grid_c[indices] - grid_c[grid_ids])

-32739.53125

In [9]:
grid_ids = np.load('../data/FIRE/grid-ids--ioni_gas.npy').astype(int)


In [16]:
np.shape(np.where(grid_ids != indices))

(1, 383327)

In [ ]:
non_empty_points = ioni_gas["density"]!=0

non_empty_points_reshaped = np.reshape(non_empty_points, voxel_resolution**3)
quantity = np.reshape(ioni_gas["density"], voxel_resolution**3)

voronoied_result = quantity[non_empty_points_reshaped][grid_ids] / np.bincount(grid_ids)[grid_ids]
voronoied_result = voronoied_result.reshape((voxel_resolution, voxel_resolution, voxel_resolution))

In [20]:
grid_batch_size   = 1000
points_batch_size = 1000

# Create batches
batch_counter = np.arange(grid_batch_size, len(grid_c), grid_batch_size)
batch_counter[-1] = len(grid_c)

points_counter = np.arange(points_batch_size, len(non_empty_points), points_batch_size)
points_counter[-1] = len(non_empty_points)

# points_counter = np.array([len(non_empty_points)])

grid_batch_intervals     = np.array([[0]+list(batch_counter[0:-1]),   list(batch_counter)]).transpose()
points_counter_intervals = np.array([[0]+list(points_counter[0:-1]), list(points_counter)]).transpose()

In [21]:
%%time
print(">> calculating all batch combinations...")
all_combinations = np.array(list(product(grid_batch_intervals, points_counter_intervals)))

>> calculating all batch combinations...
CPU times: user 4.27 s, sys: 1.45 s, total: 5.72 s
Wall time: 11.4 s


In [22]:
all_combinations

array([[[        0,      1000],
        [        0,      1000]],

       [[        0,      1000],
        [     1000,      2000]],

       [[        0,      1000],
        [     2000,      3000]],

       ...,

       [[134216000, 134217728],
        [    17000,     18000]],

       [[134216000, 134217728],
        [    18000,     19000]],

       [[134216000, 134217728],
        [    19000,     20363]]])

In [14]:
from itertools import product
import numpy as np
from multiprocessing import Pool

def generate_combinations(args):
    grid_batch_intervals, points_counter_intervals = args
    return np.array(list(product(grid_batch_intervals, points_counter_intervals)))


# Define the number of processes based on your system's capabilities
num_processes = 16

# Split the workload
chunk_size = len(grid_batch_intervals) // num_processes
chunks = [(grid_batch_intervals[i:i+chunk_size], points_counter_intervals) for i in range(0, len(grid_batch_intervals), chunk_size)]

# Use multiprocessing Pool to parallelize the combination generation
with Pool(processes=16) as pool:
    all_combinations = np.concatenate(pool.map(generate_combinations, chunks))

In [17]:
from itertools import product
import numpy as np
from numba import jit

# Define the function and apply the @jit decorator
@jit(nopython=True)
def generate_combinations(grid_batch_intervals, points_counter_intervals):
    result = []
    for i in range(len(grid_batch_intervals)):
        for j in range(len(points_counter_intervals)):
            result.append((grid_batch_intervals[i], points_counter_intervals[j]))
    return np.array(result)

# Time the execution of the Numba-optimized function
%time all_combinations = generate_combinations(grid_batch_intervals, points_counter_intervals)



TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1mNo implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(list(UniTuple(array(int64, 1d, A) x 2))<iv=None>)
 
There are 2 candidate implementations:
[1m   - Of which 2 did not match due to:
   Overload in function 'impl_np_array': File: numba/np/arrayobj.py: Line 5384.
     With argument(s): '(list(UniTuple(array(int64, 1d, A) x 2))<iv=None>)':[0m
[1m    Rejected as the implementation raised a specific error:
      TypingError: Failed in nopython mode pipeline (step: nopython frontend)
    [1m[1m[1mNo implementation of function Function(<intrinsic np_array>) found for signature:
     
     >>> np_array(list(UniTuple(array(int64, 1d, A) x 2))<iv=None>, none)
     
    There are 2 candidate implementations:
    [1m      - Of which 2 did not match due to:
          Intrinsic in function 'np_array': File: numba/np/arrayobj.py: Line 5358.
            With argu

In [15]:
%%time
# Use multiprocessing Pool to parallelize the combination generation
with Pool(processes=16) as pool:
    all_combinations = np.concatenate(pool.map(generate_combinations, chunks))



KeyboardInterrupt



In [5]:
all_combinations

array([[[        0,      1000],
        [        0,      1000]],

       [[        0,      1000],
        [     1000,      2000]],

       [[        0,      1000],
        [     2000,      3000]],

       ...,

       [[134216000, 134217728],
        [    17000,     18000]],

       [[134216000, 134217728],
        [    18000,     19000]],

       [[134216000, 134217728],
        [    19000,     20363]]])

In [ ]:
all_combinations_2 = np.array(np.meshgrid(grid_batch_intervals[:, 0], points_counter_intervals[:, 0],
                              grid_batch_intervals[:, 1], points_counter_intervals[:, 1])).T.reshape(-1, 2, 2)


In [14]:
np.shape(points_counter_intervals)

(20, 2)

In [12]:
all_combinations_2[1]

array([   0, 1000])

In [33]:
# len(non_empty_points)

len(all_combinations)
all_combinations[len(all_combinations)-1]

array([[134200000, 134217728],
       [    10000,     20363]])

In [43]:
all_combinations[0]

array([[   0, 1000],
       [   0, 1000]])

In [6]:
# without multithreading
for indexes in all_combinations:
    process_interval(indexes)

===> [[    0 10000]
 [    0 10000]]
===> [[    0 10000]
 [10000 20000]]
===> [[    0 10000]
 [20000 30000]]
===> [[    0 10000]
 [30000 40000]]
===> [[    0 10000]
 [40000 50000]]
===> [[    0 10000]
 [50000 60000]]
===> [[    0 10000]
 [60000 70000]]
===> [[    0 10000]
 [70000 80000]]



KeyboardInterrupt



In [ ]:
# with multithreading
print(f"grid batch max: {len(grid_c)} \t points batch max: {non_empty_points_length}")

with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_interval, indexes): indexes
               for indexes in all_combinations}


KeyboardInterrupt



In [14]:
all_combinations[0][0]

array([    0, 10000])

In [34]:
all_combinations[0:10]

array([[[     0,  10000],
        [     0,  30000]],

       [[     0,  10000],
        [ 30000,  60000]],

       [[     0,  10000],
        [ 60000,  90000]],

       [[     0,  10000],
        [ 90000, 120000]],

       [[     0,  10000],
        [120000, 150000]],

       [[     0,  10000],
        [150000, 180000]],

       [[     0,  10000],
        [180000, 210000]],

       [[     0,  10000],
        [210000, 240000]],

       [[     0,  10000],
        [240000, 270000]],

       [[     0,  10000],
        [270000, 300000]]])

In [11]:
grid_c

===> [[      0   10000]
 [1810000 1820000]]


array([[-9.98046875, -9.98046875, -9.98046875],
       [-9.98046875, -9.98046875, -9.94140625],
       [-9.98046875, -9.98046875, -9.90234375],
       ...,
       [ 9.98046875,  9.98046875,  9.90234375],
       [ 9.98046875,  9.98046875,  9.94140625],
       [ 9.98046875,  9.98046875,  9.98046875]])

===> [[      0   10000]
 [1820000 1830000]]
===> [[      0   10000]
 [1830000 1840000]]
===> [[      0   10000]
 [1840000 1850000]]
===> [[      0   10000]
 [1850000 1860000]]
===> [[      0   10000]
 [1860000 1870000]]


In [ ]:
# Process batches concurrently
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_interval, indexes): indexes
               for indexes in all_combinations}

    for future in concurrent.futures.as_completed(futures):
        indexes = futures[future]
        print(f"Working on batch {indexes}", end="\r")
        # print(f"Processed interval [{interval_min}:{interval_max}]")


# np.save(location_save_grid_ids, grid_ids)


===>===> [[     0  10000]
 [900000 910000]]
===> [[     0  10000]
 [910000 920000]]
===> [[     0  10000]
 [920000 930000]]
===> [[     0  10000]
 [930000 940000]]
===> [[     0  10000]
 [940000 950000]]
===> [[     0  10000]
 [950000 960000]]
===> [[     0  10000]
 [960000 970000]]
===> [[     0  10000]
 [970000 980000]]
===> [[     0  10000]
 [980000 990000]]
===> [[      0   10000]
 [ 990000 1000000]]
===> [[      0   10000]
 [1000000 1010000]]
===> [[      0   10000]
 [1010000 1020000]]
===> [[      0   10000]
 [1020000 1030000]]
===> [[      0   10000]
 [1030000 1040000]]
===> [[      0   10000]
 [1040000 1050000]]
===> [[      0   10000]
 [1050000 1060000]]
===> [[    0 10000]
 [    0 10000]]
===> [[    0 10000]
 [10000 20000]]
===> [[    0 10000]
 [20000 30000]]
===> [[    0 10000]
 [30000 40000]]
===> [[      0   10000]
 [1060000 1070000]]
===> [[    0 10000]
 [40000 50000]]
===> [[      0   10000]
 [1070000 1080000]]
===> [[      0   10000]
 [1080000 1090000]]
===> [[    0 100


KeyboardInterrupt



In [41]:
process_interval(indexes)

In [44]:
current_min_distances[0:10001]

array([             4,              4,              4, ...,
                    4,              4, 10240000000000])

In [38]:
with ThreadPoolExecutor() as executor:
    future = executor.submit(process_interval, indexes)
    # print(future.result())

In [59]:
# list(product(grid_batch_intervals, points_counter_intervals))

In [4]:
# Parameters
batch_size = 100

# Function to calculate distances and update grid_ids for a given interval
def process_interval(start, end):
    grid_ids[start:end] = np.argmin(distance.cdist(grid_c[start:end], non_empty_points), axis=1)

# Create an array to store grid_ids
grid_ids = np.empty(len(grid_c), dtype=int)

# Create batches
batch_counter = np.arange(batch_size, len(grid_c), batch_size)
batch_counter[-1] = len(grid_c)

In [18]:


np.argmin(distance.cdist(grid_c[start:end], non_empty_points[9:20]), axis=1)

array([7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

In [12]:
# for interval_min, interval_max in zip([0] + list(batch_counter), batch_counter):
#     print(interval_min, interval_max)

In [ ]:


# Process batches concurrently
with ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_interval, interval_min, interval_max): (interval_min, interval_max)
               for interval_min, interval_max in zip([0] + list(batch_counter), batch_counter)}

    for future in concurrent.futures.as_completed(futures):
        interval_min, interval_max = futures[future]
        print(f"Working on batch [{interval_min}-{interval_max}] out of {len(grid_c)}, so {interval_max/len(grid_c)*100}% done.", end="\r")
        # print(f"Processed interval [{interval_min}:{interval_max}]")


np.save(location_save_grid_ids, grid_ids)
